# Simulasi dan Implementasi Berbagai Filter untuk Navigasi Robot

Notebook ini mencakup implementasi Extended Kalman Filter (EKF) dan Particle Filter (PF) untuk navigasi menggunakan pengukuran dari sensor IMU dan LiDAR. Setiap bagian disimulasikan dengan noise dan memperkirakan posisi robot berdasarkan pengukuran yang ada.

## Particle Filter untuk Navigasi

Pada bagian ini, kita menggunakan Particle Filter untuk memperkirakan posisi robot. Pengukuran dari IMU digunakan untuk prediksi posisi, sementara LiDAR digunakan untuk memperbarui posisi.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt

# Simulasi parameter
np.random.seed(42)  # Seed untuk hasil yang reproduktif
time_steps = 50  # Jumlah langkah waktu
actual_velocity = 1.0  # Kecepatan sebenarnya (konstan)
imu_noise_std = 0.05  # Standar deviasi noise dari IMU
lidar_noise_std = 1.0  # Standar deviasi noise dari LiDAR
process_noise_std = 0.1  # Standar deviasi noise proses
num_particles = 100  # Jumlah partikel untuk Particle Filter

# Inisialisasi variabel untuk Particle Filter
actual_positions = [0]
imu_measurements = []
lidar_measurements = []
filtered_positions_pf = []

particles = np.random.uniform(-10, 10, num_particles)  # Posisi awal partikel
weights = np.ones(num_particles) / num_particles  # Bobot awal partikel

# Fungsi resampling partikel
def resample_particles(particles, weights):
    indices = np.random.choice(range(len(particles)), size=len(particles), p=weights)
    return particles[indices]

for t in range(1, time_steps + 1):
    # Posisi sebenarnya (dengan noise proses)
    actual_positions.append(actual_positions[-1] + actual_velocity + np.random.normal(0, process_noise_std))

    # Pengukuran dari sensor IMU (kecepatan dengan noise)
    imu_measurements.append(actual_velocity + np.random.normal(0, imu_noise_std))

    # Pengukuran dari sensor LiDAR (posisi dengan noise)
    lidar_measurements.append(actual_positions[-1] + np.random.normal(0, lidar_noise_std))

    # Particle Filter
    # Prediction step (dengan IMU untuk prediksi posisi)
    particles += imu_measurements[-1] + np.random.normal(0, process_noise_std, num_particles)

    # Update step (menggunakan pengukuran dari LiDAR)
    weights *= np.exp(-0.5 * ((lidar_measurements[-1] - particles) / lidar_noise_std) ** 2)
    weights += 1e-300  # Hindari pembagian dengan nol
    weights /= sum(weights)  # Normalisasi

    # Resample step
    particles = resample_particles(particles, weights)

    # Estimasi posisi berdasarkan rata-rata bobot partikel
    filtered_positions_pf.append(np.average(particles, weights=weights))

# Visualisasi hasil
plt.figure(figsize=(12, 8))
plt.plot(range(time_steps + 1), actual_positions, label="Posisi Sebenarnya", color="green", linewidth=2)
plt.scatter(range(1, time_steps + 1), lidar_measurements, label="Pengukuran LiDAR", color="red", alpha=0.6)
plt.plot(range(1, time_steps + 1), filtered_positions_pf, label="Posisi Setelah Filtering (Particle Filter)", color="orange", linewidth=2)
plt.xlabel("Waktu (step)")
plt.ylabel("Posisi")
plt.title("Simulasi Navigasi dengan Particle Filter")
plt.legend()
plt.grid()
plt.show()
    